In [4]:
import pandas as pd
import numpy as np
from tqdm import tqdm

In [8]:
tmps = []
for i in tqdm(range(81)):
    tmp = pd.read_csv('./data/test/{}.csv'.format(i))
    tmps.append(tmp)
    
test = pd.concat(tmps, axis=0).reset_index(drop=True)

100%|█████████████████████████████████████████████████████████████████████████████████| 81/81 [00:00<00:00, 434.08it/s]


In [16]:
test[:340]

,Day,Hour,Minute,DHI,DNI,WS,RH,T,TARGET,file
0,0,0,0,0,0,2.7,34.42,0.0,0.0,0
1,0,0,30,0,0,2.7,34.17,0.1,0.0,1
2,0,1,0,0,0,2.7,34.23,0.2,0.0,2
3,0,1,30,0,0,2.7,33.99,0.3,0.0,3
4,0,2,0,0,0,2.8,33.97,0.4,0.0,4
...,...,...,...,...,...,...,...,...,...,...
335,6,23,30,0,0,3.4,51.96,-8.4,0.0,335
336,0,0,0,0,0,2.8,70.34,-6.1,0.0,336
337,0,0,30,0,0,2.9,71.97,-6.4,0.0,337
338,0,1,0,0,0,2.9,69.77,-6.7,0.0,338


In [39]:
len(test)

27216

In [26]:
test['file'] = test.index
test['file'] = (test['file']/336).astype(int)
test['date'] = test['Day'] + 7*test['file']

In [29]:
test

,Day,Hour,Minute,DHI,DNI,WS,RH,T,TARGET,file,date
0,0,0,0,0,0,2.7,34.42,0.0,0.0,0,0
1,0,0,30,0,0,2.7,34.17,0.1,0.0,0,0
2,0,1,0,0,0,2.7,34.23,0.2,0.0,0,0
3,0,1,30,0,0,2.7,33.99,0.3,0.0,0,0
4,0,2,0,0,0,2.8,33.97,0.4,0.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
27211,6,21,30,0,0,0.8,63.35,13.7,0.0,80,566
27212,6,22,0,0,0,0.7,64.82,13.1,0.0,80,566
27213,6,22,30,0,0,0.7,66.10,12.8,0.0,80,566
27214,6,23,0,0,0,0.6,67.64,12.4,0.0,80,566


In [41]:
def time_minute(data):
    hour = data['Hour']
    if data['Minute'] == 0:
        minute = 0
    else:
        minute = 0.5
    
    return hour + minute

In [42]:
ghi = test.iloc[np.where(test['DHI'] + test['DNI'] >0)[0]].reset_index(drop=True)
ghi['time_float'] = [time_minute(ghi.iloc[i]) for i in range(len(ghi.index))]

In [44]:
test_by_date = pd.DataFrame(ghi.groupby(['date'])['time_float'].min()).reset_index()
test_by_date.rename(columns = {'time_float': 'sunrise'}, inplace=True)
test_by_date['sunset'] = pd.DataFrame(ghi.groupby(['Day'])['time_float'].max())['time_float']
test_by_date['sunnyTime'] = test_by_date['sunset'] - test_by_date['sunrise']

In [45]:
test_by_date

,date,sunrise,sunset,sunnyTime
0,0,7.5,19.5,12.0
1,1,7.5,19.5,12.0
2,2,7.5,19.5,12.0
3,3,7.5,19.5,12.0
4,4,7.5,19.5,12.0
...,...,...,...,...
562,562,4.5,NaN,NaN
563,563,4.5,NaN,NaN
564,564,4.5,NaN,NaN
565,565,4.5,NaN,NaN


In [46]:
test_by_date['T_mean'] = pd.DataFrame(test.groupby(['date'])['T'].mean())['T']
test_by_date['DHI_mean'] = pd.DataFrame(test.groupby(['date'])['DHI'].mean())['DHI']
test_by_date['DNI_mean'] = pd.DataFrame(test.groupby(['date'])['DNI'].mean())['DNI']
test_by_date['WS_mean'] = pd.DataFrame(test.groupby(['date'])['WS'].mean())['WS']
test_by_date['RH_mean'] = pd.DataFrame(test.groupby(['date'])['RH'].mean())['RH']
test_by_date['TARGET_mean'] = pd.DataFrame(test.groupby(['date'])['TARGET'].mean())['TARGET']

In [47]:
test_by_date

,date,sunrise,sunset,sunnyTime,T_mean,DHI_mean,DNI_mean,WS_mean,RH_mean,TARGET_mean
0,0,7.5,19.5,12.0,4.652083,45.312500,84.395833,2.556250,35.827292,6.703528
1,1,7.5,19.5,12.0,5.450000,40.395833,78.166667,1.454167,39.045208,6.214770
2,2,7.5,19.5,12.0,3.414583,38.083333,121.583333,1.185417,44.850625,7.370306
3,3,7.5,19.5,12.0,0.512500,22.229167,156.229167,1.616667,56.891042,7.507346
4,4,7.5,19.5,12.0,-2.258333,16.125000,317.333333,1.839583,49.736667,11.253597
...,...,...,...,...,...,...,...,...,...,...
562,562,4.5,NaN,NaN,14.922917,107.312500,274.875000,1.414583,47.613958,25.421188
563,563,4.5,NaN,NaN,15.972917,85.770833,326.041667,1.627083,51.993333,27.129276
564,564,4.5,NaN,NaN,18.062500,108.479167,291.604167,1.772917,49.728333,25.689886
565,565,4.5,NaN,NaN,17.520833,108.520833,229.416667,1.304167,48.447292,23.092370


In [48]:
X = test_by_date.drop('date', axis=1)

In [50]:
X

,sunrise,sunset,sunnyTime,T_mean,DHI_mean,DNI_mean,WS_mean,RH_mean,TARGET_mean
0,7.5,19.5,12.0,4.652083,45.312500,84.395833,2.556250,35.827292,6.703528
1,7.5,19.5,12.0,5.450000,40.395833,78.166667,1.454167,39.045208,6.214770
2,7.5,19.5,12.0,3.414583,38.083333,121.583333,1.185417,44.850625,7.370306
3,7.5,19.5,12.0,0.512500,22.229167,156.229167,1.616667,56.891042,7.507346
4,7.5,19.5,12.0,-2.258333,16.125000,317.333333,1.839583,49.736667,11.253597
...,...,...,...,...,...,...,...,...,...
562,4.5,NaN,NaN,14.922917,107.312500,274.875000,1.414583,47.613958,25.421188
563,4.5,NaN,NaN,15.972917,85.770833,326.041667,1.627083,51.993333,27.129276
564,4.5,NaN,NaN,18.062500,108.479167,291.604167,1.772917,49.728333,25.689886
565,4.5,NaN,NaN,17.520833,108.520833,229.416667,1.304167,48.447292,23.092370


## 모델 불러와서 예측하기

In [51]:
import joblib

In [53]:
model = joblib.load('month_xgb_200120.pkl')
predict_values = model.predict(X)

In [56]:
test_by_date['month'] = predict_values

In [58]:
test_by_date['month'].unique()

array([ 2,  3,  0,  1, 11, 10,  9,  8], dtype=int64)

In [59]:
test_by_date['month'].value_counts()

9     258
10    109
11    104
8      89
3       2
1       2
0       2
2       1
Name: month, dtype: int64

In [61]:
test_by_date[:50]

,date,sunrise,sunset,sunnyTime,T_mean,DHI_mean,DNI_mean,WS_mean,RH_mean,TARGET_mean,month
0,0,7.5,19.5,12.0,4.652083,45.312500,84.395833,2.556250,35.827292,6.703528,2
1,1,7.5,19.5,12.0,5.450000,40.395833,78.166667,1.454167,39.045208,6.214770,3
2,2,7.5,19.5,12.0,3.414583,38.083333,121.583333,1.185417,44.850625,7.370306,3
3,3,7.5,19.5,12.0,0.512500,22.229167,156.229167,1.616667,56.891042,7.507346,0
4,4,7.5,19.5,12.0,-2.258333,16.125000,317.333333,1.839583,49.736667,11.253597,0
5,5,7.5,19.5,12.0,-1.810417,27.625000,193.187500,1.202083,64.367917,8.706162,1
6,6,7.5,19.5,12.0,-5.487500,32.000000,8.333333,4.258333,65.420833,3.280946,1
7,7,7.5,NaN,NaN,-7.387500,40.604167,101.541667,3.029167,50.174167,7.160091,11
8,8,7.5,NaN,NaN,-9.289583,41.604167,73.250000,1.866667,56.062292,6.401682,11
9,9,7.5,NaN,NaN,-6.152083,14.520833,325.500000,2.633333,46.826458,11.353767,11
